In [1]:
import os

from optimizer.io_utils import DataLoader
from optimizer.viz_utils import save_fp_recall_plots

data_path = os.path.join('data', 
                         'production_alerts_meta_data.csv'
            )

# Let's open the original dataframe.
# This function extracts the camera_id's and maps the theft labels as theft and generates a binary 'is_theft' collumn.
df = DataLoader(data_path, source_type="csv").load()

num_stores = df['store'].nunique()
num_cameras = df.groupby(['store', 'camera_id']).ngroups
num_theft_events = df['is_theft'].sum()
num_non_theft_events = (df['is_theft'] == 0).sum()

print(
    f"Summary: "
    f"Number of unique videos: {num_stores}, "
    f"Number of unique cameras: {num_cameras}, "
    f"Theft events: {num_theft_events}, "
    f"Non-theft events: {num_non_theft_events}"
)

# Now let's export the FP = f(recall) plots for each pair store, camera.
# For writting this function I've used sklearn's precision_recall_curve and tracked #FP
# using (probability >= th).astype(int) and comparing to the 'true' value  'is_theft'.
# The results are stored in 'plots/store_camera_id.png'
#save_fp_recall_plots(df, output_dir="plots/")

/Users/nfsrules/Documents/veesion/challenge-veesion/challenge-veesion-venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Summary: Number of unique videos: 45, Number of unique cameras: 45, Theft events: 1601, Non-theft events: 154336


In [4]:
cam

{'store': 'be-ad-1420-hugo-3',
 'camera_id': 10,
 'threshold': 0.22448979591836732,
 'fp_saved': 509,
 'tp_lost': 1}

In [14]:
import json
from optimizer import CameraModel, MultiCameraOptimizer

# Load optimization results
with open("results/optim.json", "r") as f:
    optim = json.load(f)

# Normalize df columns for matching
df["store"] = df["store"].astype(str).str.strip().str.lower()
df["camera_id"] = df["camera_id"].astype(str).str.strip()

# Group after normalization
grouped = df.groupby(["store", "camera_id"])

# Final totals
total_tp_lost = 0
total_fp_saved = 0

# Process all cameras in optimization result
for cam in optim.get("cameras_info", []):
    store = str(cam.get("store", "")).strip().lower()
    camera_id = str(cam.get("camera_id", "")).strip()
    threshold = cam.get("threshold")
    fp_saved = cam.get("fp_saved", 0)
    tp_lost = cam.get("tp_lost", 0)

    group_key = (store, camera_id)

    if group_key in grouped.groups:
        group = grouped.get_group(group_key)
        X = group["probability"].values.astype(float)
        y = group["is_theft"].values

        camera = CameraModel(camera_id=camera_id, store=store)
        camera.threshold = threshold
        camera.eval(X, y, threshold=threshold)

        total_tp_lost += tp_lost
        total_fp_saved += fp_saved

        print(f"[{store} | Camera {camera_id}] ✅ Evaluated | TP lost: {tp_lost}, FP saved: {fp_saved}")
    else:
        print(f"[{store} | Camera {camera_id}] ⚠️  No matching data in df")

# Final summary
print(f"\n🔚 Final Summary: Total TP Lost = {total_tp_lost}, Total FP Saved = {total_fp_saved}")


[be-ad-1420-hugo-3 | Camera 10] ✅ Evaluated | TP lost: 1, FP saved: 509
[be-carr-1030-plasky-88 | Camera 6] ✅ Evaluated | TP lost: 0, FP saved: 4
[be-carr-6000-waterloo-60 | Camera 13] ✅ Evaluated | TP lost: 0, FP saved: 106
[be-inter-7500-eisenhower-14 | Camera 12] ✅ Evaluated | TP lost: 9, FP saved: 2982
[be-proxy-1000-haute-221-227 | Camera 25] ✅ Evaluated | TP lost: 0, FP saved: 3
[ci-ivoire-abidjan-98 | Camera 22] ✅ Evaluated | TP lost: 1, FP saved: 897
[cz-dm-15000-plzenska-8 | Camera 6] ✅ Evaluated | TP lost: 24, FP saved: 6261
[es-char-30161-mayor-199 | Camera 17] ✅ Evaluated | TP lost: 1, FP saved: 2058
[es-char-46021-ros-16 | Camera 14] ✅ Evaluated | TP lost: 0, FP saved: 249
[es-condis-08912-guifre-100 | Camera 7] ✅ Evaluated | TP lost: 0, FP saved: 113
[es-mas-33207-aviles-6 | Camera 3] ✅ Evaluated | TP lost: 2, FP saved: 1011
[es-spar-35480-alamo-1 | Camera 215] ✅ Evaluated | TP lost: 0, FP saved: 272
[fr-carr-13001-canebiere-80 | Camera 7] ✅ Evaluated | TP lost: 5, FP sav

In [12]:
import json
from optimizer import CameraModel, MultiCameraOptimizer

# Load optimization results
with open("results/optim.json", "r") as f:
    optim = json.load(f)

grouped = df.groupby(["store", "camera_id"])

# To collect final results
total_tp_lost = 0
total_fp_saved = 0

for cam in optim.get("cameras_info", []):
    store = cam.get("store")
    camera_id = cam.get("camera_id")
    threshold = cam.get("threshold")
    fp_saved = cam.get("fp_saved")
    tp_lost = cam.get("tp_lost")

    # Instantiate camera model and set threshold
    camera = CameraModel(camera_id=camera_id, store=store)
    camera.threshold = threshold

    # Get matching group from df
    group_key = (store, camera_id)
    if group_key in grouped.groups:
        group = grouped.get_group(group_key)
        X = group["probability"].values.astype(float)
        y = group["is_theft"].values
        camera.eval(X, y, threshold=threshold)

        # Optionally accumulate results
        total_tp_lost += tp_lost
        total_fp_saved += fp_saved

        print(f"[{store} | Camera {camera_id}] ✅ Evaluated | Threshold: {threshold:.3f}, TP lost: {tp_lost}, FP saved: {fp_saved}")
    else:
        print(f"[{store} | Camera {camera_id}] ⚠️  No matching data in df")

# Print final totals
print(f"\n🔚 Final Summary: Total TP Lost = {total_tp_lost}, Total FP Saved = {total_fp_saved}")


[be-ad-1420-hugo-3 | Camera 10] ⚠️  No matching data in df
[be-carr-1030-plasky-88 | Camera 6] ⚠️  No matching data in df
[be-carr-6000-waterloo-60 | Camera 13] ⚠️  No matching data in df
[be-inter-7500-eisenhower-14 | Camera 12] ⚠️  No matching data in df
[be-proxy-1000-haute-221-227 | Camera 25] ⚠️  No matching data in df
[ci-ivoire-abidjan-98 | Camera 22] ⚠️  No matching data in df
[cz-dm-15000-plzenska-8 | Camera 6] ⚠️  No matching data in df
[es-char-30161-mayor-199 | Camera 17] ⚠️  No matching data in df
[es-char-46021-ros-16 | Camera 14] ⚠️  No matching data in df
[es-condis-08912-guifre-100 | Camera 7] ⚠️  No matching data in df
[es-mas-33207-aviles-6 | Camera 3] ⚠️  No matching data in df
[es-spar-35480-alamo-1 | Camera 215] ⚠️  No matching data in df
[fr-carr-13001-canebiere-80 | Camera 7] ⚠️  No matching data in df
[fr-carr-22560-chardons | Camera 12] ⚠️  No matching data in df
[fr-carr-61190-juchereau | Camera 11] ⚠️  No matching data in df
[fr-carr-69530-gaulle-213 | Camer

In [7]:
import json
from optimizer import CameraModel, MultiCameraOptimizer

with open("results/optim.json", "r") as f:
    optim = json.load(f)

for cam in optim.get("cameras_info"):
    break

store = cam.get("store")
camera_id = cam.get("camera_id")
threshold = cam.get("threshold")
fp_saved = cam.get("fp_saved")
tp_lost = cam.get('tp_lost')


camera = CameraModel(camera_id=camera_id, store=store)
camera.threshold = threshold

grouped = df.groupby(["store", "camera_id"])

for (store_, cam_id_), group in grouped:
    if store_ == store:
        print("ok!")
        X = group["probability"].values.astype(float)
        y = group["is_theft"].values
        camera.eval(X, y, threshold=threshold)
        break
    

ok!


In [10]:
camera._compute_tp_fp(X, y, threshold=threshold)

(np.int64(28), np.int64(1579))

{'store': 'be-ad-1420-hugo-3',
 'camera_id': '10',
 'threshold': 0.22448979591836732,
 'baseline_tp': 29,
 'baseline_fp': 2087,
 'eval_tp': 28,
 'eval_fp': 1579,
 'tp_lost': 1,
 'fp_saved': 508}